In [1]:
import clinical_trial_extractor
import SQLConnect

In [2]:
# Get mapping from project to their publications pmids, along with
# list of all publications records
data = clinical_trial_extractor.extract()
project_pub = {
    (datum['title'], datum['start_date']): [pub[0] for pub in datum['works']]
    for datum in data
}
publications = []
for key, val in project_pub.items():
    publications += val

In [3]:
# Upload publications
publication_queries = [SQLConnect.UnmergedV1.publication(("ClinicalTrials.gov", None, rec)) for rec in publications]
# SQLConnect.connect_and_query(queries=publication_queries, types=["INSERT" for _ in publication_queries], database="UnmergedV1")

In [4]:
# Get our publication IDs
pub_id_queries = ["SELECT pmid, pub_id FROM Publication WHERE origin_database='ClinicalTrials.gov'"]
pub_id = SQLConnect.connect_and_query(queries=pub_id_queries, types=["SELECT"], database="UnmergedV1")[0]
pub_id = {
    rec[0]: rec[1]
    for rec in pub_id
}

Connection to database established
MySQL connection is closed


In [5]:
# Get mapping from projects to our project ids
query = 'SELECT project_id, title, start_date FROM Project WHERE origin_database="ClinicalTrials.gov"'
project_data = SQLConnect.connect_and_query(queries=[query], types=['SELECT'], database="UnmergedV1")[0]
project_to_id = {
    (record[1], str(record[2])): record[0]
    for record in project_data
}

Connection to database established
MySQL connection is closed


In [6]:
# Map project id to their publications ids, and make list of querys from them
project_pub_id = {}
project_not_found = set()
for key, val in project_pub.items():
    if key in project_to_id:
        project_pub_id[project_to_id[key]] = [
            pub_id[pmid] for pmid in val
        ]
    else:
        project_not_found.add(key)
project_pub_query = [
    SQLConnect.UnmergedV1.project_pub((key, idx))
    for key, val in project_pub_id.items()
    for idx in val
]

In [7]:
print(len(project_not_found))
len(project_pub)

22


4023

In [8]:
# SQLConnect.connect_and_query(project_pub_query, ["INSERT" for _ in project_pub_query], 'UnmergedV1')

In [10]:
work_upload_queries = [(None, id) for id in pub_id.values()]
work_upload_queries = [SQLConnect.UnmergedV1.work(entry) for entry in work_upload_queries]
SQLConnect.connect_and_query(work_upload_queries, ["INSERT" for _ in work_upload_queries], 'UnmergedV1')

Connection to database established
MySQL connection is closed


[]